# Segmenting and Clustering Neighbourhoods in Toronto

## Introduction

In this notebook, I convert a collection of Toronto addresses (obtained via web parsing) into their equivalent latitude and longitude values using a geocoding API. Secondly, I use the Foursquare API to explore neighbourhoods in Toronto. Then I use the explore function to get the most common venue categories in each neighbourhood, and additionally use this feature to group the neighbourhoods into clusters. To compute these clusters, the k-means clustering algorithm will be used. Finally, I use the Folium library to visualize the neighbourhoods in Toronto and their emerging clusters.

## Part 1: Web Parsing Addresses from Wikipedia
#### Source: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
postal_codes = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header=0)[0]
postal_codes.rename(columns={"Neighborhood": "Neighbourhood"}, inplace=True)
postal_codes.head() # Display first 5 rows

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


#### Size of retrieved dataframe from the Wikipedia article:

In [3]:
postal_codes.shape

(287, 3)

#### We ignore entries with an unassigned borough, and replace any unassigned neighbourhood values with their respective borough

In [4]:
postal_codes = postal_codes[postal_codes['Borough'] != 'Not assigned'] 

def fill_unassigned_neighbourhoods(row): # Function to fill in any unassigned neighbourhood values with their borough
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
    return row

postal_codes.apply(fill_unassigned_neighbourhoods, axis=1)
postal_codes.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


#### Since more than one neighbourhood can exist in one postal code area, any of these relevant rows will be combined into one row with the neighbourhoods comma-separated as shown in the following table

In [5]:
postal_codes = postal_codes.groupby(['Postcode', 'Borough'])['Neighbourhood'].agg([('Neighbourhood', ', '.join)]).reset_index()
postal_codes.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### Size of cleaned dataframe:

In [6]:
postal_codes.shape

(103, 3)

## Part 2: Adding Latitude and Longitude Coordinates

In [7]:
import geocoder

In [8]:
lats = []
lons = []

for postal_code in postal_codes['Postcode']:
    lat_lng_coords = None
    # loop until we get the coordinates
    while lat_lng_coords is None:
      g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
      lat_lng_coords = g.latlng

    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    lats.append(latitude)
    lons.append(longitude)

#### Now that we have the collection of latitudes and longitudes in `lats` and `lons`, we can add them as columns to the `postal_codes` dataframe

In [9]:
postal_codes['Latitude'] = lats
postal_codes['Longitude'] = lons
postal_codes.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


#### Renaming `postal_codes` dataframe variable to a more suitable one

In [10]:
toronto_data = postal_codes

## Part 3: Segmenting and Clustering

#### Defining Foursquare API Credentials and Version

In [11]:
CLIENT_ID = '4VPWXU0OLOV2JJDGVKMQFZREWFNZUZEWIQFLH0OJTPF2MGRW'
CLIENT_SECRET = 'GWL2IKBB51O5WTLOIK4IPZI5NILY4IKN52WZA4CACSOF1WBW'
VERSION = '20200114' # Foursquare API version

#### We get the top 100 venues that are in each neighbourhood within a radius of 500 meters

In [12]:
import requests # For retrieving JSON data from the Foursquare API
radius = 500
LIMIT = 100

In [13]:
# Function to accomplish this
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return nearby_venues

#### We create a new dataframe to store the venue data for each neighbourhood

In [14]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

In [15]:
print(toronto_venues.shape)
toronto_venues.head()

(2466, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.811525,-79.195517,Straits Aqualife Limited,43.809268,-79.199399,Pet Store
1,"Rouge, Malvern",43.811525,-79.195517,Canadian Appliance Source Whitby,43.808353,-79.191331,Home Service
2,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Chris Effects Painting,43.784343,-79.163742,Construction & Landscaping
3,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,Royal Canadian Legion,43.782533,-79.163085,Bar
4,"Guildwood, Morningside, West Hill",43.765815,-79.175193,Homestead Roofing Repair,43.765140,-79.178663,Construction & Landscaping


#### We check how many venues were returned for each neighbourhood

In [16]:
toronto_venues.groupby('Neighbourhood').size().to_frame('Total Venues').reset_index()

,Neighbourhood,Total Venues
0,"Adelaide, King, Richmond",100
1,Agincourt,14
2,"Agincourt North, L'Amoreaux East, Milliken, St...",2
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",15
4,"Alderwood, Long Branch",4
5,"Bathurst Manor, Downsview North, Wilson Heights",1
6,Bayview Village,3
7,"Bedford Park, Lawrence Manor East",22
8,Berczy Park,62
9,"Birch Cliff, Cliffside West",6


#### How many of these venue categories are unique?

In [17]:
len(toronto_venues['Venue Category'].unique())

266

### Time to analyze each neighbourhood!

In [18]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighbourhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighbourhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# display resulting dataframe
toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### We have changed the venue category column's values from a categorical one into individual numerical columns where 1 represents yes and 0 represents no

#### For example: the first entry of Rouge, Malvern only has a Pet Store as its venue category (since every other column value is a 0)

In [19]:
toronto_onehot.shape

(2466, 267)

#### Now we group the rows by neighbourhood and by taking the mean of the frequency of occurrence for each venue category

In [20]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Carpet Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Fireworks Store,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hong Kong Restaurant,Hookah Bar,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Climbing Spot,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stationery Store,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.0,0.03,0.0,0.01,0.0,0.03,0.0,0.0,0.0,0.0,0.0,0.0,0.020000,0.0,0.03,0.0,0.0,0.0,0.01,0.000000,0.0,0.0,0.0,0.0,0.02,0.0,0.01,0.03,0.0,0.0,0.0,0.

In [21]:
toronto_grouped.shape

(99, 267)

#### We can create a dataframe containing the top 10 most common venues for each neighbourhood

In [22]:
# Function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [23]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd'] # suffixes for words like 1st, 2nd, etc.

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Hotel,Gym,American Restaurant,Breakfast Spot,Bar,Burger Joint,Asian Restaurant
1,Agincourt,Shopping Mall,Chinese Restaurant,Discount Store,Park,Sushi Restaurant,Vietnamese Restaurant,Supermarket,Hong Kong Restaurant,Skating Rink,Grocery Store
2,"Agincourt North, L'Amoreaux East, Milliken, St...",Pharmacy,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
3,"Albion Gardens, Beaumond Heights, Humbergate, ...",Grocery Store,Pizza Place,Coffee Shop,Fast Food Restaurant,Pharmacy,Liquor Store,Caribbean Restaurant,Fried Chicken Joint,Sandwich Place,Beer Store
4,"Alderwood, Long Branch",Convenience Store,Dance Studio,Pub,Gym,Fish Market,Fish & Chips Shop,Fireworks Store,Flea Market,Field,Electronics Store


In [24]:
neighbourhoods_venues_sorted.shape

(99, 11)

### The fun part! Clustering!

In [25]:
import folium

#### We first display a map centered at the center of Toronto (43.6532° N, 79.3832° W) with all of the neighbourhoods

In [26]:
center = [43.6532, -79.3832] 

map_neighbourhoods = folium.Map(location=center, zoom_start=11) # Initialize the map

for index, row in toronto_data.iterrows(): # Add the neighbourhoods
    folium.Marker([row['Latitude'], row['Longitude']],
        popup=row['Neighbourhood']
    ).add_to(map_neighbourhoods)
    
map_neighbourhoods # Display the map. Note that this will not show on Github

#### Now for the actual clustering using the k-means clustering algorithm!

In [27]:
from sklearn.cluster import KMeans

In [28]:
# set number of clusters to 4 arbitrarily (could experiment with different values)
kclusters = 4

# cluster based on the 10th most common venues for each neighbourhood
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1) 

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

#### We create a dataframe that includes the cluster labeling as well as the top 10 most common venues for each neighbourhood

In [29]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

# a small number of neighbourhoods have no data so we will drop these
toronto_merged = toronto_merged.dropna()

# convert cluster labels column to int from float
toronto_merged['Cluster Label'] = toronto_merged['Cluster Label'].astype(int)

# adding 1 to each label (so that Cluster 1 is the first cluster and so on instead of Cluster 0 as the first)
toronto_merged['Cluster Label'] = toronto_merged['Cluster Label'] + 1

toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1B,Scarborough,"Rouge, Malvern",43.811525,-79.195517,1,Home Service,Pet Store,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785665,-79.158725,1,Bar,Construction & Landscaping,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765815,-79.175193,3,Park,Construction & Landscaping,Bus Stop,Gym / Fitness Center,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
3,M1G,Scarborough,Woburn,43.768369,-79.217590,3,Soccer Field,Park,Business Service,Coffee Shop,Korean Restaurant,Yoga Studio,Farm,Ethiopian Restaurant,Event Space,Falafel Restaurant
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440,4,Playground,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market


In [30]:
toronto_merged.shape

(100, 16)

#### Now to map out the k-means clusters!

In [31]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [32]:
# create map
map_clusters = folium.Map(location=center, zoom_start=11)

# set colour scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Label']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

# display resulting map with clusters
map_clusters

## Part 4: Examining the Clusters
#### We can examine each cluster and potentially determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, we could then assign a name to each cluster

#### Cluster 1 (which happens to be the largest one!)

In [33]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Scarborough,1,Home Service,Pet Store,Yoga Studio,Fast Food Restaurant,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
1,Scarborough,1,Bar,Construction & Landscaping,Yoga Studio,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Field
5,Scarborough,1,Grocery Store,Train Station,Restaurant,Indian Restaurant,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Yoga Studio
6,Scarborough,1,Discount Store,Convenience Store,Department Store,Coffee Shop,Yoga Studio,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant
7,Scarborough,1,Bus Line,Bakery,Intersection,Bus Station,Soccer Field,Coffee Shop,Metro Station,Farmers Market,Event Space,Falafel Restaurant
8,Scarborough,1,Discount Store,Coffee Shop,Bistro,Gift Shop,Bank,Sandwich Place,Pharmacy,Fireworks Store,Field,Fast Food Restaurant
9,Scarborough,1,College Stadium,Skating Rink,General Entertainment,Gym Pool,Park,Gym,Elementary School,Empanada Restaurant,Ethiopian Restaurant,Event Space
10,Scarborough,1,Light Rail Station,Construction & Landscaping,Gift Shop,Bakery,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
12,Scarborough,1,Shopping Mall,Chinese Restaurant,Discount Store,Park,Sushi Restaurant,Vietnamese Restaurant,Supermarket,Hong Kong Restaurant,Skating Rink,Grocery Store
13,Scarborough,1,Pharmacy,Pizza Place,Golf Course,Coffee Shop,Chinese Restaurant,Shopping Mall,Fried Chicken Joint,Thai Restaurant,Hobby Shop,Grocery Store


#### Cluster 2 (the 4th largest)

In [34]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
94,Etobicoke,2,Golf Course,Yoga Studio,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant,Electronics Store


#### Cluster 3 (the 2nd largest)

In [35]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Scarborough,3,Park,Construction & Landscaping,Bus Stop,Gym / Fitness Center,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
3,Scarborough,3,Soccer Field,Park,Business Service,Coffee Shop,Korean Restaurant,Yoga Studio,Farm,Ethiopian Restaurant,Event Space,Falafel Restaurant
19,North York,3,Construction & Landscaping,Park,Trail,Yoga Studio,Farmers Market,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
23,North York,3,Convenience Store,Speakeasy,Bank,Park,Yoga Studio,Farmers Market,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm
25,North York,3,Food & Drink Shop,Fireworks Store,Park,Flea Market,Fish Market,Fish & Chips Shop,Flower Shop,Field,Electronics Store,Fast Food Restaurant
26,North York,3,Burger Joint,Gas Station,Construction & Landscaping,Coffee Shop,Park,Yoga Studio,Farm,Ethiopian Restaurant,Event Space,Falafel Restaurant
30,North York,3,Food Court,Airport,Coffee Shop,Park,Fast Food Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
34,North York,3,Grocery Store,German Restaurant,Park,Yoga Studio,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market
40,East York,3,Farmers Market,Bar,Italian Restaurant,Park,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
41,East Toronto,3,Grocery Store,Bus Line,Discount Store,Park,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market


#### Cluster 4 (3rd largest)

In [36]:
toronto_merged.loc[toronto_merged['Cluster Label'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Scarborough,4,Playground,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Farmers Market
11,Scarborough,4,Auto Garage,Convenience Store,Yoga Studio,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
48,Central Toronto,4,Playground,Convenience Store,Gym,Flea Market,Fish Market,Fish & Chips Shop,Fireworks Store,Field,Fast Food Restaurant,Eastern European Restaurant
89,Etobicoke,4,Convenience Store,Dance Studio,Pub,Gym,Fish Market,Fish & Chips Shop,Fireworks Store,Flea Market,Field,Electronics Store
